In [2]:
import pandas as pd
import json
from pandas import Series, DataFrame
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Reformatting data in movies and credits table

In [3]:
def load_tmdb_movies(path):
    df = pd.read_csv(path)
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.date())
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df


def load_tmdb_credits(path):
    df = pd.read_csv(path)
    json_columns = ['cast', 'crew']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

In [4]:
movies = load_tmdb_movies("tmdb_5000_movies.csv")
credits = load_tmdb_credits("tmdb_5000_credits.csv")

In [5]:
def safe_access(container, index_values):
    
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan


credits['gender_of_lead'] = credits.cast.apply(lambda x: safe_access(x, [0, 'gender']))
credits['lead'] = credits.cast.apply(lambda x: safe_access(x, [0, 'name']))

In [6]:
df = pd.merge(movies, credits, left_on='id', right_on='movie_id')
df[['original_title', 'revenue', 'lead', 'gender_of_lead']].sort_values(by=['revenue'], ascending=False)[:10]

,original_title,revenue,lead,gender_of_lead
0,Avatar,2787965087,Sam Worthington,2.0
25,Titanic,1845034188,Kate Winslet,1.0
16,The Avengers,1519557910,Robert Downey Jr.,2.0
28,Jurassic World,1513528810,Chris Pratt,2.0
44,Furious 7,1506249360,Vin Diesel,2.0
7,Avengers: Age of Ultron,1405403694,Robert Downey Jr.,2.0
124,Frozen,1274219009,Kristen Bell,1.0
31,Iron Man 3,1215439994,Robert Downey Jr.,2.0
546,Minions,1156730962,Sandra Bullock,1.0
26,Captain America: Civil War,1153304495,Chris Evans,2.0


In [7]:
credits.apply(lambda row: [x.update({'movie_id': row['movie_id']}) for x in row['cast']], axis=1);
credits.apply(lambda row: [x.update({'movie_id': row['movie_id']}) for x in row['crew']], axis=1);
credits.apply(lambda row: [person.update({'order': order}) for order, person in enumerate(row['crew'])], axis=1);

cast = []
credits.cast.apply(lambda x: cast.extend(x))
cast = pd.DataFrame(cast)
cast['type'] = 'cast'

crew = []
credits.crew.apply(lambda x: crew.extend(x))
crew = pd.DataFrame(crew)
crew['type'] = 'crew'

cast_crew = pd.concat([cast, crew],  ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [8]:
cast_crew.head()

,cast_id,character,credit_id,department,gender,id,job,movie_id,name,order,type
0,242.0,Jake Sully,5602a8a7c3a3685532001c9a,NaN,2,65731,NaN,19995,Sam Worthington,0,cast
1,3.0,Neytiri,52fe48009251416c750ac9cb,NaN,1,8691,NaN,19995,Zoe Saldana,1,cast
2,25.0,Dr. Grace Augustine,52fe48009251416c750aca39,NaN,1,10205,NaN,19995,Sigourney Weaver,2,cast
3,4.0,Col. Quaritch,52fe48009251416c750ac9cf,NaN,2,32747,NaN,19995,Stephen Lang,3,cast
4,5.0,Trudy Chacon,52fe48009251416c750ac9d3,NaN,1,17647,NaN,19995,Michelle Rodriguez,4,cast


In [9]:
credits.head()

,movie_id,title,cast,crew,gender_of_lead,lead
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...",2.0,Sam Worthington
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",2.0,Johnny Depp
2,206647,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de...",2.0,Daniel Craig
3,49026,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de...",2.0,Christian Bale
4,49529,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...",2.0,Taylor Kitsch


In [10]:
movies.apply(lambda row: [x.update({'id': row['id']}) for x in row['keywords']], axis=1)
keywords = []
movies.keywords.apply(lambda x: keywords.extend(x))
keywords = pd.DataFrame(keywords)
keywords.head()

,id,name
0,19995,culture clash
1,19995,future
2,19995,space war
3,19995,space colony
4,19995,society


In [11]:
movies.apply(lambda row: [x.update({'id': row['id']}) for x in row['genres']], axis=1)
genres = []
movies.genres.apply(lambda x: genres.extend(x))
genres = pd.DataFrame(genres)
genres.head()

,id,name
0,19995,Action
1,19995,Adventure
2,19995,Fantasy
3,19995,Science Fiction
4,285,Adventure


In [12]:
movies.apply(lambda row: [x.update({'id': row['id']}) for x in row['production_companies']], axis=1)
production_companies = []
movies.production_companies.apply(lambda x: production_companies.extend(x))
production_companies = pd.DataFrame(production_companies)
production_companies.head()

,id,name
0,19995,Ingenious Film Partners
1,19995,Twentieth Century Fox Film Corporation
2,19995,Dune Entertainment
3,19995,Lightstorm Entertainment
4,285,Walt Disney Pictures


In [13]:
movies.apply(lambda row: [x.update({'id': row['id']}) for x in row['production_countries']], axis=1)
production_countries = []
movies.production_countries.apply(lambda x: production_countries.extend(x))
production_countries = pd.DataFrame(production_countries)
production_countries.head()

,id,iso_3166_1,name
0,19995,US,United States of America
1,19995,GB,United Kingdom
2,285,US,United States of America
3,206647,GB,United Kingdom
4,206647,US,United States of America


In [14]:
movies=df[['id','original_title', 'revenue','popularity','tagline','vote_average','vote_count', 'lead', 'gender_of_lead']]

In [15]:
movies.head()

,id,original_title,revenue,popularity,tagline,vote_average,vote_count,lead,gender_of_lead
0,19995,Avatar,2787965087,150.437577,Enter the World of Pandora.,7.2,11800,Sam Worthington,2.0
1,285,Pirates of the Caribbean: At World's End,961000000,139.082615,"At the end of the world, the adventure begins.",6.9,4500,Johnny Depp,2.0
2,206647,Spectre,880674609,107.376788,A Plan No One Escapes,6.3,4466,Daniel Craig,2.0
3,49026,The Dark Knight Rises,1084939099,112.312950,The Legend Ends,7.6,9106,Christian Bale,2.0
4,49529,John Carter,284139100,43.926995,"Lost in our world, found in another.",6.1,2124,Taylor Kitsch,2.0


Now, we have reformatted the tables with the below structure:
Movies table: Movie id, movie title, revenue, popularity, tagline, vote_average, vote_count, lead, gender of lead
Cast Crew table: Movie id, cast id, character, credit_id, department, gender,id, job, name of actor, order of appearance, type(cast/crew)
Keywords table: Movie id, Keywords for each movie id
Genres table: Movie id, Genres for each movie id
Production companies table: Movie id, Production companies for each movie id
Production countries table: Movie id, Production countries for each movie id


Differentatiating between animation and non-animation movies

In [16]:
pd.merge(cast_crew[cast_crew['job']=='Animation'],movies, left_on='movie_id',right_on='id').head()

,cast_id,character,credit_id,department,gender,id_x,job,movie_id,name,order,type,id_y,original_title,revenue,popularity,tagline,vote_average,vote_count,lead,gender_of_lead
0,NaN,NaN,54c379219251412446005555,Visual Effects,0,1417836,Animation,49026,Andrew McEvoy,156,crew,49026,The Dark Knight Rises,1084939099,112.312950,The Legend Ends,7.6,9106,Christian Bale,2.0
1,NaN,NaN,55345dd3c3a368523e0028c1,Visual Effects,0,1456835,Animation,99861,Anthony Rizzo,61,crew,99861,Avengers: Age of Ultron,1405403694,134.279229,A New Age Has Come.,7.3,6767,Robert Downey Jr.,2.0
2,NaN,NaN,5523f0ddc3a36828d100689e,Visual Effects,0,1355894,Animation,57201,Anthony Di Ninno,29,crew,57201,The Lone Ranger,89289910,49.046956,Never Take Off the Mask,5.9,2311,Johnny Depp,2.0
3,NaN,NaN,553155c8c3a3682217000563,Visual Effects,0,1452991,Animation,2454,Daniel Fotheringham,68,crew,2454,The Chronicles of Narnia: Prince Caspian,419651413,53.978602,Hope has a new face.,6.3,1630,Ben Barnes,2.0
4,NaN,NaN,55495951c3a36841b20008a6,Visual Effects,0,1459736,Animation,24428,Travis Tohill,145,crew,24428,The Avengers,1519557910,144.448633,Some assembly required.,7.4,11776,Robert Downey Jr.,2.0


In [17]:
df_animation=pd.merge(genres[genres['name']=='Animation'],movies, left_on='id',right_on='id')


In [18]:
df_nonanimation=pd.merge(genres[genres['name']!='Animation'],movies, left_on='id',right_on='id')

In [19]:
len(df_animation)/len(movies)*100.0

4.871955028107433

Using Animation within jobs under crew gives ambiguos results, such as The Dark Knight Rises, Avengers etc, which are not animation movies. A better method to get animation movies is by using Animation within genres table. We can see that out of all the total number of movies, 4.87% movies are animation movies

Relationship between revenue, production companies, genres, gender of lead

In [20]:
movies_prod=pd.merge(movies[['id','original_title','revenue','gender_of_lead']],\
                     production_companies,left_on='id',right_on='id')\
               .merge(genres,right_on='id',left_on='id')

In [21]:
movies_prod.head()

,id,original_title,revenue,gender_of_lead,name_x,name_y
0,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Action
1,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Adventure
2,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Fantasy
3,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Science Fiction
4,19995,Avatar,2787965087,2.0,Twentieth Century Fox Film Corporation,Action


Relationship between revenue, gender of the lead, production house type, genres, popularity, vote count

In [22]:
movies_prod['bigprodhouse']=0
movies_prod['bigprodhouse'][movies_prod['name_x'].isin(movies_prod['name_x'].value_counts().loc[lambda x:x>100].index.values)]=1
movies_prod.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,original_title,revenue,gender_of_lead,name_x,name_y,bigprodhouse
0,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Action,0
1,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Adventure,0
2,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Fantasy,0
3,19995,Avatar,2787965087,2.0,Ingenious Film Partners,Science Fiction,0
4,19995,Avatar,2787965087,2.0,Twentieth Century Fox Film Corporation,Action,1


In [23]:
movies_all=movies_prod.merge(movies,right_on='id',left_on='id')
movies_all.head()
import statsmodels.api as sm
from patsy import dmatrices
y, X = dmatrices('revenue_x ~ C(gender_of_lead_x)+C(bigprodhouse)+C(name_y)+popularity+vote_count',\
                 data=movies_all, return_type='dataframe')

In [24]:
model = sm.OLS(y, X)    
result = model.fit()       
print (result.summary())

                            OLS Regression Results                            
Dep. Variable:              revenue_x   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.641
Method:                 Least Squares   F-statistic:                     2658.
Date:                Sat, 31 Aug 2019   Prob (F-statistic):               0.00
Time:                        16:06:43   Log-Likelihood:            -7.1336e+05
No. Observations:               35741   AIC:                         1.427e+06
Df Residuals:                   35716   BIC:                         1.427e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

Created a regression model to analyze the relationship between revenue and production house type, popularity, vote count and genres. From the above table, we can see that 64% of revenue can be accurately predicted using these parameters. For a high revenue movie, it looks like the best combination would be a movie under a big production house with a male lead, having  high popularity and vote count. The genres that garner the most revenue seem to be animation, adventure, family and fantasy. 